In [19]:
import general.utils 
def convertSED(name,gt,pe):
    import pandas as pd
    label_set = ['alarm', 'crying baby', 'crash', 'barking dog', 'running engine', 'female scream', \
                'female speech', 'burning fire', 'footsteps', 'knocking on door', 'male scream', 'male speech', \
                    'ringing phone', 'piano']
    
    import datatool.seddata
    dataset=datatool.seddata.SED(gt)
    dataset.load()
    ######
    from datetime import datetime
    run_date=datetime.now().strftime('%y%m%d_%H-%M-%S')
    run_info={'dataset':'SED2020','run_date':run_date,'dataset_path':'', 'strategy':'EIN','evalution':'-'}
    ######
    pred=datatool.seddata.SED(pe)
    pred.load()
    
    ########
    evalres={0:{'test':general.utils.Data('SED')}}
    
    evalres[0]['test'].real_events=dataset.activity_events
    evalres[0]['test'].Sdata=None
    evalres[0]['test'].predicted=None
    evalres[0]['test'].shortrunname=name
    evalres[0]['test'].predicted_classes=None
    evalres[0]['test'].event_cm=None
    evalres[0]['test'].quality={'accuracy': 0, 'precision': 0, 'recall': 0, 'f1': 0}
    evalres[0]['test'].pred_events=pred.activity_events

    #######
    general.utils.saveState([run_info,dataset,{0:evalres[0]}],name)
convertSED('sed-ein1','/workspace/sed/gt1.csv','/workspace/sed/pe1.csv')

In [20]:
    import  general.utils as utils
    import result_analyse.resultloader
    import result_analyse.visualisation as vs

    from ipywidgets import interact, interactive, fixed, interact_manual,widgets
    import numpy as np
    import pandas as pd

    @interact
    def result_selector(file=result_analyse.resultloader.get_runs()):
        if(file==None):return
        print('Analysing ',file)
        run_info,dataset,evalres=utils.loadState(file)
        stime=dataset.activity_events.iloc[0].StartTime
        #etime=stime+np.timedelta64(1,'D')
        etime=dataset.activity_events.iloc[-1].EndTime

        for i in range(len(evalres)):
                quality=evalres[i]['test'].quality
                print('Evalution quality fold=%d is %s' % (i, quality))
        print(len(dataset.sensor_events))
        
    #     vs.plot_CM(dataset,evalres)
        
        @interact
        def viewFold(fold= range(len(evalres))):
            @interact_manual
            def view(start_date=widgets.DatePicker(value=pd.to_datetime(stime)),end_date=widgets.DatePicker(value=pd.to_datetime(etime)),debug=widgets.Checkbox(value=False)):
                print(evalres[fold]['test'].real_events)
                duration=(pd.to_datetime(start_date),pd.to_datetime(end_date))
                duration2=(pd.to_datetime(start_date),pd.to_datetime(start_date)+pd.DateOffset(days=7))
                real_events=vs.filterTime(evalres[fold]['test'].real_events,duration)
                pred_events=vs.filterTime(evalres[fold]['test'].pred_events,duration)
                vs.plotJoinAct(dataset,real_events,pred_events)
                acts=[p for p in dataset.activities_map]
                labels=[dataset.activities_map[p] for p in acts]
                print(acts)
                print(labels)
                vs.plotJoinAct2(real_events,pred_events,acts,labels,duration=duration2)
                #vs.plot_per_act(dataset,{'test':evalres})
                
                from matplotlib import pyplot as plt
                plt.rc_context(rc={'figure.max_open_warning': 0})
                import result_analyse.SpiderChart
                result_analyse.SpiderChart.radar_factory(5, frame='polygon')
                acount=len(dataset.activities_map)
                a_fig,a_ax=plt.subplots(acount-1,1,figsize=(10, acount*.25),)
    #             a_fig.tight_layout(pad=3.0)
                col=4        
                row=int(np.ceil((acount-1.0)/float(col)))
                m_fig,m_ax=plt.subplots(row,col,figsize=(col*3, row*3),subplot_kw=dict(projection='radar'))
                if type(a_ax)!=np.ndarray:
                    print('dddd',a_ax)
                    print(type(a_ax))
                    a_ax=np.array([a_ax])
                else:
                    m_ax=m_ax.flatten()
                for i in range(acount-1,len(m_ax)):
                    m_ax[i].set_visible(False)

                for i in range(1,len(dataset.activities_map)):
    #                 real_events2,pred_events2=vs.remove_gaps(real_events,pred_events,i)
                    #real_events2,pred_events2=vs.remove_gaps(real_events,pred_events,i,max_events=10)
                    real_events2,pred_events2=real_events,pred_events
                    vs.plotJoinAct(dataset,real_events,pred_events,onlyAct=i,ax=a_ax[i-1])
                    try:
    #                     vs.plotJoinAct(dataset,real_events2,pred_events2,onlyAct=i,ax=a_ax[i-1])
                        
                        vs.plotMyMetric2(dataset,real_events2,pred_events2,onlyAct=i,ax=m_ax[i-1],debug=debug,calcne=0)
    
                    except Exception as e:
                        import sys
                        import traceback
                        print(e, file=sys.stderr)
                        traceback.print_exc()

                    #    print('error in ',i)
                    #vs.plotWardMetric(dataset,real_events,pred_events,onlyAct=i)
    #             vs.plotJoinMyMetric(dataset,real_events2,pred_events2,calcne=0)
                # a_fig.show()
                m_fig.tight_layout(pad=0,h_pad=-20.0, w_pad=3.0)
                # m_fig.show()

    #             @interact
    #             def view2(onlyAct=[(str(i)+" : "+dataset.activities[i],i)for i in dataset.activities_map]):
    #                 print(onlyAct)
    #                 #vs.visualize(dataset)
    #                 # vs.my_result_analyse(evalres[i].real_events,evalres[i].pred_events)               
    #                 #vs.plotJoinAct(dataset,real_events,pred_events,onlyAct=onlyAct)
    #                 real_events2,pred_events2=vs.remove_gaps(real_events,pred_events,onlyAct)
    #                 vs.plotJoinAct(dataset,real_events2,pred_events2,onlyAct=onlyAct)
    #                 vs.plotMyMetric(dataset,real_events,pred_events,onlyAct=onlyAct)
    #                 vs.plotWardMetric(dataset,real_events,pred_events,onlyAct=onlyAct)


interactive(children=(Dropdown(description='file', options=(('sed-ein1', 'sed-ein1'), ('sed-ein', 'sed-ein'), …

In [2]:
import general.utils
df=general.utils.loadState('cache/aaa')

In [3]:
df

,existance,total dur.,relative dur.,alignment,cardinality
tp,76.000000,303.846000,69.438862,64.331483,72.000000
fp,8.000000,33.720000,10.264140,3.772385,17.000000
fn,24.000000,176.214000,30.561138,7.896131,28.000000
precision,0.904762,0.900108,0.871220,0.944608,0.808989
recall,0.760000,0.632933,0.694389,0.890677,0.720000
f1,0.826087,0.743240,0.772818,0.916850,0.761905


In [4]:
calcf1=2*df.loc['recall']*df.loc['precision']/(df.loc['recall']+df.loc['precision'])
calcf1

existance        0.826087
total dur.       0.743240
relative dur.    0.772818
alignment        0.916850
cardinality      0.761905
dtype: float64

In [5]:
df.loc['f2']=calcf1
df

,existance,total dur.,relative dur.,alignment,cardinality
tp,76.000000,303.846000,69.438862,64.331483,72.000000
fp,8.000000,33.720000,10.264140,3.772385,17.000000
fn,24.000000,176.214000,30.561138,7.896131,28.000000
precision,0.904762,0.900108,0.871220,0.944608,0.808989
recall,0.760000,0.632933,0.694389,0.890677,0.720000
f1,0.826087,0.743240,0.772818,0.916850,0.761905
f2,0.826087,0.743240,0.772818,0.916850,0.761905


In [51]:
!pip install --upgrade pandas

     |████████████████████████████████| 9.5MB 6.1MB/s eta 0:00:01     |██████████████████████▉         | 6.8MB 6.1MB/s eta 0:00:01
ERROR: wrangle 0.6.7 has requirement scipy==1.2, but you'll have scipy 1.3.2 which is incompatible.
ERROR: tsfresh 0.12.0 has requirement pandas<=0.23.4,>=0.20.3, but you'll have pandas 1.1.5 which is incompatible.
ERROR: chartify 2.6.1 has requirement pandas<1.0.0,>=0.21.0, but you'll have pandas 1.1.5 which is incompatible.
ERROR: alibi 0.3.1 has requirement tensorflow<2.0, but you'll have tensorflow 2.2.1 which is incompatible.
  Found existing installation: pandas 0.25.3
    Uninstalling pandas-0.25.3:
      Successfully uninstalled pandas-0.25.3
